### Intel MobileODT Kaggle competition

https://www.kaggle.com/c/intel-mobileodt-cervical-cancer-screening


In [1]:
import os, sys
from __future__ import print_function, division
from importlib import reload 
import utils_p3; reload(utils_p3)
from utils_p3 import *
%matplotlib inline
from IPython.display import FileLink
import tensorflow as tf
import six
import numpy as np
import pandas as pd
import cv2
import glob
import random
from PIL import ImageFile
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
#print("TensorFlow version: %s" % tf.__version__)
print("Theano version: %s" % theano.__version__)
print("Keras version: %s" % keras.__version__)

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:01:00.0)
Using Theano backend.


Theano version: 0.9.0
Keras version: 2.0.3


### Global declarations

In [2]:
HOME_DIR = os.getcwd()
#path = "data/imgs/"
path = "data/testing/"
#path = "data/testing/sample/"
train_valid_fraction = 0.75
image_shape = (224,224)
patience = 3
batch_size = 64
ImageFile.LOAD_TRUNCATED_IMAGES = True
keras.backend.image_data_format() #verify image_data_format for theano vs TF

'channels_first'

In [3]:
%pwd

'/media/eric/SSD500/fastai/deeplearning1/nbs/MobileODT'

In [4]:
HOME_DIR

'/media/eric/SSD500/fastai/deeplearning1/nbs/MobileODT'

### Setup batches

In [5]:
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(path+'test', batch_size=batch_size*2, shuffle=False)
steps_per_epoch = int(np.ceil(batches.samples/batch_size))
validation_steps = int(np.ceil(val_batches.samples/(batch_size*2)))

Found 5048 images belonging to 3 classes.
Found 1683 images belonging to 3 classes.
Found 4018 images belonging to 1 classes.


In [6]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 5048 images belonging to 3 classes.
Found 1683 images belonging to 3 classes.
Found 4018 images belonging to 1 classes.


## Imagenet conv features with VGG16
from Statefarm_original notebook

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all.

So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)

In [7]:
# Import our class
import vgg16_p3; reload(vgg16_p3)
from vgg16_p3 import Vgg16

### Step 1: pre-compute the output of the last convolutional layer

In [8]:
# Grab VGG16 and find the last convolutional layer
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [9]:
# Build a new model that includes everything up to that last convolutional layer
conv_model = Sequential(conv_layers)

In [ ]:
# Predict the outputs of that model by calculating the activations of that last convolutional layer
conv_feat = conv_model.predict_generator(batches, int(np.ceil(batches.samples/batch_size)), workers=3)

In [ ]:
# As this takes time, save it to load it in the future
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
conv_val_feat = conv_model.predict_generator(val_batches, int(np.ceil(val_batches.samples/(batch_size*2))), workers=3)

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)

In [ ]:
conv_test_feat = conv_model.predict_generator(test_batches, int(np.ceil(test_batches.samples/(batch_size*2))), workers=3)

In [ ]:
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [ ]:
# Data Augmentation: at 1:15:55 in L4 video, we can see Jeremy had a cell for gen_t,
# then computed a da_conv_feat with nb_sample*5

In [ ]:
# We can look at the original model and find the last convo layer "conv2d_13" with output shape (none, 512, 14, 14)
# and compare it with the shape of of our new model's output.
model.summary()

In [ ]:
# It's the same than "conv2d_13" !
conv_val_feat.shape

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_test_feat = load_array(path+'results/conv_test_feat.dat')
conv_val_feat.shape

### Step 2: Build new model on top, with dense layers
Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 10 classes. Let's try using a simplified version of VGG's dense layers.

In [ ]:
# we make 'p' a parameter to try different Dropout amounts
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(3, activation='softmax')
        ]

In [ ]:
p=0.5

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
p=0.8
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, epochs=50, 
             validation_data=(conv_val_feat, val_labels))

### Step 3: Pre-computed data augmentation by making 5 augmented copies of training set

We'll use our usual data augmentation parameters:

In [ ]:
# potential DA: warning uses HUGE amount of RAM and takes like 90 mins for a simple batches.samples*2, check forum.
# http://forums.fast.ai/t/state-farm-full-how-not-to-run-out-of-memory-with-vgg-da-batches-samples-5/3469/2
# maybe reduce batch_size below to 1 ? YES !!!
gen_t = image.ImageDataGenerator(rotation_range=8, height_shift_range=0.025,
                                shear_range=0.05, channel_shift_range=10, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=1, shuffle=False)

In [ ]:
da_conv_feat = conv_model.predict_generator(da_batches, int(np.ceil(da_batches.samples*5)), workers=4)

In [ ]:
save_array(path+'results/da_5_conv_feat.dat', da_conv_feat)

In [ ]:
da_conv_feat = load_array(path+'results/da_5_conv_feat.dat')

Let's include the real training data as well in its non-augmented form.

In [20]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [ ]:
da_conv_feat_update = np.concatenate([da_conv_feat, conv_feat])

In [ ]:
save_array(path+'results/da_5_conv_feat_update.dat', da_conv_feat_update)

In [10]:
da_conv_feat_update = load_array(path+'results/da_5_conv_feat_update.dat')

Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.

In [12]:
da_trn_labels = np.concatenate([trn_labels]*6)

In [13]:
save_array(path+'results/da_trn_labels.dat', da_trn_labels)

In [15]:
da_trn_labels = load_array(path+'results/da_trn_labels.dat')

Based on some experiments the previous model works well, maybe with bigger dense layers like 512 later ?

In [26]:
# we make 'p' a parameter to try different Dropout amounts and we take out flatten here
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(3, activation='softmax')
        ]

In [27]:
p=0.6

In [28]:
bn_da_model = Sequential(get_bn_da_layers(p))
bn_da_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/1
30288/30288 [==============================] - 34s - loss: 1.3651 - acc: 0.6119 - val_loss: 1.6018 - val_acc: 0.5663


In [30]:
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=50, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/50
30288/30288 [==============================] - 31s - loss: 0.6745 - acc: 0.7694 - val_loss: 1.7393 - val_acc: 0.5977
Epoch 2/50
30288/30288 [==============================] - 17s - loss: 0.5183 - acc: 0.8312 - val_loss: 1.8474 - val_acc: 0.6031
Epoch 3/50
30288/30288 [==============================] - 15s - loss: 0.4609 - acc: 0.8556 - val_loss: 2.0051 - val_acc: 0.6173
Epoch 4/50
30288/30288 [==============================] - 15s - loss: 0.3789 - acc: 0.8814 - val_loss: 1.7808 - val_acc: 0.6227
Epoch 5/50
30288/30288 [==============================] - 16s - loss: 0.3336 - acc: 0.8961 - val_loss: 2.1367 - val_acc: 0.5995
Epoch 6/50
30288/30288 [==============================] - 16s - loss: 0.2922 - acc: 0.9072 - val_loss: 2.0435 - val_acc: 0.6191
Epoch 7/50
30288/30288 [==============================] - 16s - loss: 0.2819 - acc: 0.9145 - val_loss: 1.9085 - val_acc: 0.6370
Epoch 8/50
30288/30288 [==============================]

In [31]:
# Let's save those weights.
bn_da_model.save_weights(path+'models/bn_da5_conv2.h5')

In [ ]:
# Let's load those weights.
bn_da_model.load_weights(path+'models/bn_da5_conv2.h5')

#### some test not saved with different learning rates or optimizer

In [50]:
bn_da_model.compile(RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/25
30288/30288 [==============================] - 12s - loss: 0.2520 - acc: 0.9455 - val_loss: 2.6079 - val_acc: 0.6423
Epoch 2/25
30288/30288 [==============================] - 13s - loss: 0.2131 - acc: 0.9491 - val_loss: 2.4361 - val_acc: 0.6376
Epoch 3/25
30288/30288 [==============================] - 13s - loss: 0.1904 - acc: 0.9513 - val_loss: 2.3996 - val_acc: 0.6393
Epoch 4/25
30288/30288 [==============================] - 13s - loss: 0.1887 - acc: 0.9515 - val_loss: 2.2177 - val_acc: 0.6387
Epoch 5/25
30288/30288 [==============================] - 13s - loss: 0.1765 - acc: 0.9507 - val_loss: 2.1010 - val_acc: 0.6441
Epoch 6/25
30288/30288 [==============================] - 13s - loss: 0.1566 - acc: 0.9559 - val_loss: 1.9740 - val_acc: 0.6387
Epoch 7/25
30288/30288 [==============================] - 13s - loss: 0.1462 - acc: 0.9558 - val_loss: 2.0105 - val_acc: 0.6453
Epoch 8/25
30288/30288 [==============================]

In [61]:
from keras.optimizers import *
bn_da_model.compile(Adagrad(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/25
30288/30288 [==============================] - 14s - loss: 0.0889 - acc: 0.9705 - val_loss: 1.7762 - val_acc: 0.6465
Epoch 2/25
30288/30288 [==============================] - 13s - loss: 0.0852 - acc: 0.9727 - val_loss: 1.7887 - val_acc: 0.6471
Epoch 3/25
30288/30288 [==============================] - 13s - loss: 0.0898 - acc: 0.9702 - val_loss: 1.7650 - val_acc: 0.6488
Epoch 4/25
30288/30288 [==============================] - 13s - loss: 0.0915 - acc: 0.9704 - val_loss: 1.7723 - val_acc: 0.6482
Epoch 5/25
30288/30288 [==============================] - 12s - loss: 0.0854 - acc: 0.9714 - val_loss: 1.7676 - val_acc: 0.6471
Epoch 6/25
30288/30288 [==============================] - 13s - loss: 0.0848 - acc: 0.9719 - val_loss: 1.7625 - val_acc: 0.6477
Epoch 7/25
30288/30288 [==============================] - 13s - loss: 0.0901 - acc: 0.9705 - val_loss: 1.7565 - val_acc: 0.6465
Epoch 8/25
30288/30288 [==============================]

In [62]:
bn_da_model.compile(Adadelta(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/25
30288/30288 [==============================] - 20s - loss: 0.0816 - acc: 0.9738 - val_loss: 1.7224 - val_acc: 0.6471
Epoch 2/25
30288/30288 [==============================] - 20s - loss: 0.0885 - acc: 0.9710 - val_loss: 1.7166 - val_acc: 0.6500
Epoch 3/25
30288/30288 [==============================] - 21s - loss: 0.0877 - acc: 0.9705 - val_loss: 1.7280 - val_acc: 0.6477
Epoch 4/25
30288/30288 [==============================] - 20s - loss: 0.0892 - acc: 0.9713 - val_loss: 1.7299 - val_acc: 0.6488
Epoch 5/25
30288/30288 [==============================] - 20s - loss: 0.0842 - acc: 0.9725 - val_loss: 1.7126 - val_acc: 0.6482
Epoch 6/25
30288/30288 [==============================] - 20s - loss: 0.0861 - acc: 0.9718 - val_loss: 1.7198 - val_acc: 0.6488
Epoch 7/25
30288/30288 [==============================] - 20s - loss: 0.0903 - acc: 0.9700 - val_loss: 1.7308 - val_acc: 0.6477
Epoch 8/25
30288/30288 [==============================]

In [63]:
bn_da_model.compile(Adamax(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/25
30288/30288 [==============================] - 15s - loss: 0.0804 - acc: 0.9723 - val_loss: 1.7197 - val_acc: 0.6376
Epoch 2/25
30288/30288 [==============================] - 15s - loss: 0.0852 - acc: 0.9719 - val_loss: 1.7375 - val_acc: 0.6453
Epoch 3/25
30288/30288 [==============================] - 16s - loss: 0.0845 - acc: 0.9723 - val_loss: 1.7081 - val_acc: 0.6441
Epoch 4/25
30288/30288 [==============================] - 15s - loss: 0.0888 - acc: 0.9708 - val_loss: 1.6761 - val_acc: 0.6417
Epoch 5/25
30288/30288 [==============================] - 15s - loss: 0.0830 - acc: 0.9726 - val_loss: 1.6833 - val_acc: 0.6506825 - acc: 0.9
Epoch 6/25
30288/30288 [==============================] - 15s - loss: 0.0858 - acc: 0.9707 - val_loss: 1.6899 - val_acc: 0.6447
Epoch 7/25
30288/30288 [==============================] - 15s - loss: 0.0823 - acc: 0.9720 - val_loss: 1.6834 - val_acc: 0.6488
Epoch 8/25
30288/30288 [=================

In [64]:
bn_da_model.compile(Nadam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_da_model.fit(da_conv_feat_update, da_trn_labels, batch_size=batch_size, epochs=25, 
             validation_data=(conv_val_feat, val_labels))

Train on 30288 samples, validate on 1683 samples
Epoch 1/25
30288/30288 [==============================] - 17s - loss: 0.0744 - acc: 0.9753 - val_loss: 1.6122 - val_acc: 0.6471
Epoch 2/25
30288/30288 [==============================] - 18s - loss: 0.0754 - acc: 0.9747 - val_loss: 1.7508 - val_acc: 0.6405
Epoch 3/25
30288/30288 [==============================] - 19s - loss: 0.0761 - acc: 0.9744 - val_loss: 1.6363 - val_acc: 0.6393
Epoch 4/25
30288/30288 [==============================] - 19s - loss: 0.0754 - acc: 0.9754 - val_loss: 1.7040 - val_acc: 0.6471
Epoch 5/25
30288/30288 [==============================] - 19s - loss: 0.0720 - acc: 0.9757 - val_loss: 1.7250 - val_acc: 0.6322
Epoch 6/25
30288/30288 [==============================] - 19s - loss: 0.0701 - acc: 0.9756 - val_loss: 1.7820 - val_acc: 0.6340
Epoch 7/25
30288/30288 [==============================] - 19s - loss: 0.0703 - acc: 0.9769 - val_loss: 1.7268 - val_acc: 0.6423
Epoch 8/25
30288/30288 [==============================]

### Submissions

Don't forget to add clipping for Kaggle submissions as it's very important to get the best cross_entropy loss function.

In [32]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [33]:
val_preds = bn_da_model.predict(conv_val_feat, batch_size=batch_size*2)

In [34]:
np.mean(keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval())

1.3345426389780415

In [35]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [36]:
preds = bn_da_model.predict(conv_test_feat, batch_size=batch_size*2)

In [37]:
subm = do_clip(preds,0.93)

In [38]:
subm_name = path+'results/subm_18_da_vgg.csv'

In [39]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [40]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image_name', [a[8:] for a in test_filenames])
submission.head()

,image_name,Type_1,Type_2,Type_3
0,0.jpg,0.007778,0.930000,0.009059
1,1.jpg,0.007778,0.019907,0.930000
2,10.jpg,0.007778,0.930000,0.007778
3,100.jpg,0.007778,0.930000,0.007778
4,10000.jpg,0.007778,0.930000,0.007778


In [41]:
submission.to_csv(subm_name, index=False)

In [42]:
FileLink(subm_name)

/media/eric/SSD500/fastai/deeplearning1/nbs/MobileODT/data/testing/results/subm_18_da_vgg.csv